In [1]:
import gdown

##Changes here in v2
## Use this code to download the testing folder. You must include these 2 lines without any changes
test_folder_id = '1LZIhiV9l82W4fNpfaBHyGvh8XBX6SoU2'
gdown.download_folder(id=test_folder_id, output="test_folder", quiet=False, use_cookies=False)

model_folder_id = "1XK3T_-hU5WLMcqlrLZe1zWOZkbiGU1tZ"
gdown.download_folder(id=model_folder_id, output="model", quiet=False, use_cookies=False)


Retrieving folder contents


Retrieving folder 1qO-cQtRdp-hLu6LMQ68OEv0zA791puJO test_images
Processing file 1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH Kek Lapis.jpg
Processing file 1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4 Kuih Kaswi Pandan.jpg
Processing file 11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6 Kuih Ketayap.jpg
Processing file 1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM Kuih Lapis.jpg
Processing file 17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB- Kuih Seri Muka.png
Processing file 1MEhhjAoBK66wtj6Xs1orsrA2vR9qLjhR Kuih Talam.jpg
Processing file 1KmI2JRpz4DnCAmPbCWoK31BJBH4ax8i7 Kuih Ubi Kayu.jpg
Processing file 1QFZdtM4ok-vH0kPgA7gdZQzT1aSppWcF Onde-Onde.jpg
Processing file 12ufiwhl6whVp33F05RLiDRieRSN13XlP test_labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kek Lapis.jpg
100%|██████████| 1.27M/1.27M [00:00<00:00, 2.25MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Kaswi Pandan.jpg
100%|██████████| 418k/418k [00:00<00:00, 1.15MB/s]
Downloading...
From: https://drive.google.com/uc?id=11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Ketayap.jpg
100%|██████████| 373k/373k [00:00<00:00, 843kB/s]
Downloading...
From: https://drive.google.com/uc?id=1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Lapis.jpg
100%|██████████| 56.6k/56

Processing file 1_7BS5j_Q45__GDzONhUppopYOuzLJi1d best.pt
Processing file 1m2-noy59PXXE5uK9AoOwYW2b9KknO_4z submission.ipynb


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1_7BS5j_Q45__GDzONhUppopYOuzLJi1d
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\model\best.pt
100%|██████████| 20.9M/20.9M [00:19<00:00, 1.06MB/s]
Downloading...
From: https://drive.google.com/uc?id=1m2-noy59PXXE5uK9AoOwYW2b9KknO_4z
To: c:\Users\zanyi\Documents\GitHub\NAIC\submission\model\submission.ipynb
100%|██████████| 10.6k/10.6k [00:00<00:00, 9.15MB/s]
Download completed


['model\\best.pt', 'model\\submission.ipynb']

In [2]:
from ultralytics import YOLO

model = YOLO("model/best.pt")

In [3]:
#test the data
import os
import pandas as pd
import csv

folderPath = r"test_folder\test_images"
store = []
for path in os.listdir(folderPath):
    filePath = os.path.join(folderPath,path)
    prediction = model.predict(filePath)
    logits = prediction[0].names[prediction[0].probs.top1]
    print(f"prediction of this is: {logits}")
    store.append([path, logits])
    
print(store)

df = pd.DataFrame(store)
with open("test_labels.txt", 'w', newline='') as f:
    f.write("Filename,Class" + '\n')
    df.to_csv(f,header=False, quotechar='"', quoting=csv.QUOTE_ALL,index=False)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\zanyi\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




image 1/1 c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kek Lapis.jpg: 640x640 Kek Lapis 1.00, Kuih Seri Muka 0.00, Kuih Ubi Kayu 0.00, Kuih Lapis 0.00, Kuih Ketayap 0.00, 7.9ms
Speed: 22.0ms preprocess, 7.9ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
prediction of this is: Kek Lapis

image 1/1 c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Kaswi Pandan.jpg: 640x640 Kuih Kaswi Pandan 0.66, Kuih Lapis 0.31, Kuih Seri Muka 0.02, Kuih Ubi Kayu 0.01, Kuih Ketayap 0.00, 6.9ms
Speed: 15.1ms preprocess, 6.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
prediction of this is: Kuih Kaswi Pandan

image 1/1 c:\Users\zanyi\Documents\GitHub\NAIC\submission\test_folder\test_images\Kuih Ketayap.jpg: 640x640 Kuih Ketayap 1.00, Kuih Ubi Kayu 0.00, Kek Lapis 0.00, Kuih Lapis 0.00, Kuih Seri Muka 0.00, 6.6ms
Speed: 28.5ms preprocess, 6.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
p